# PCA statistical arbitrage
Em dang test truoc tren S&P 500 roi sau do qua VN sau
Key idea, long some stock and short SPY
have a stock pool ( representing SPY) , use PCA on that to extract residuals, then apply ou method to see which one is undervalued ( in stat meaning) ( which has low s-score)
Buy them --> short SPY

In [ ]:
# Import Libraries
from statsmodels.tsa.ar_model import AutoReg
import pandas as pd
import numpy as np
from sklearn.decomposition import PCA
import yfinance as yf
import statsmodels.api as sm
from typing import List, Dict, Tuple


In [81]:
class PCA_StatArb_Strategy:
    def __init__(
        self,
        tickers: List[str],
        start_date: str,
        end_date: str,
        initial_balance: float = 1000000,
        fee_rate: float = 0.0043,  # 0.43% of trading value
        margin_spy: float = 0.25,  # 25% margin for SPY short
        pca_window: int = 252,
        ou_window: int = 60,
        n_factors: int = 15,
        allocation_per_stock: float = 0.05,
        beta_window: int = 252,
    ):
        self.tickers = tickers + ['SPY']
        self.start_date = start_date
        self.end_date = end_date
        self.initial_balance = initial_balance
        self.fee_rate = fee_rate
        self.margin_spy = margin_spy
        self.pca_window = pca_window
        self.ou_window = ou_window
        self.n_factors = n_factors
        self.allocation_per_stock = allocation_per_stock
        self.beta_window = beta_window

        # State variables
        self.cash = initial_balance
        self.positions: Dict[str, int] = {ticker: 0 for ticker in self.tickers}
        self.entry_prices: Dict[str, float] = {ticker: 0 for ticker in self.tickers}
        self.margin_posted = 0
        self.asset_list = []  # Portfolio state
        self.trading_log = []  # Trading actions
        self.data = None
        self.returns = None
        self.betas = None
        self.residuals = None
        self.ou_params = None
        self.signals = None
        self.pca_metrics = {}  # Store PCA performance metrics
        self.standardized_returns = None  # To store rolling standardized returns

    def fetch_data(self) -> Tuple[pd.DataFrame, pd.DataFrame]:
        """Fetch and return closing prices and returns DataFrames for stocks and SPY."""
        data = yf.download(self.tickers, start=self.start_date, end=self.end_date)['Close']
        data = data.dropna()
        returns = data.pct_change().dropna()
        self.data = data
        self.returns = returns
        return data, returns

    def compute_standardized_returns(self) -> pd.DataFrame:
        """Compute rolling standardized returns using a 60-day window."""
        returns_stocks = self.returns.drop(columns=['SPY'])
        rolling_mean = returns_stocks.rolling(window=60).mean().shift(1)  # Avoid look-ahead bias
        rolling_std = returns_stocks.rolling(window=60).std().shift(1)
        standardized_returns = (returns_stocks - rolling_mean) / rolling_std
        self.standardized_returns = standardized_returns.dropna()
        return self.standardized_returns

    def calculate_betas(self) -> pd.DataFrame:
        """Calculate and return rolling betas for each stock relative to SPY."""
        spy_returns = self.returns['SPY']
        betas = pd.DataFrame(index=self.returns.index, columns=[t for t in self.tickers if t != 'SPY'])
        for stock in betas.columns:
            stock_returns = self.returns[stock]
            rolling_cov = stock_returns.rolling(window=self.beta_window).cov(spy_returns)
            rolling_var_spy = spy_returns.rolling(window=self.beta_window).var()
            betas[stock] = rolling_cov / rolling_var_spy
        self.betas = betas.dropna()
        return self.betas

    def compute_pca_factors(self) -> Tuple[pd.Series, pd.DataFrame, pd.DataFrame, pd.DataFrame]:
        """Compute PCA residuals, explained variance, loadings, and eigenvalues for stock returns (excluding SPY)."""
        returns_stocks = self.returns.drop(columns=['SPY'])  # Exclude SPY
        self.compute_standardized_returns()  # Compute standardized returns
        n = self.pca_window
        residuals = pd.DataFrame(index=returns_stocks.index, columns=returns_stocks.columns)
        explained_var = pd.Series(index=returns_stocks.index[n-1:], dtype=float)
        eigenvalues_df = pd.DataFrame(index=returns_stocks.index[n-1:], columns=[f'Eigen_{i+1}' for i in range(self.n_factors)])
        loadings_list = []

        for t in range(n, len(returns_stocks)):
            window = self.standardized_returns.iloc[t - n:t]  # Use standardized returns for PCA
            pca = PCA(n_components=self.n_factors)
            pca.fit(window)
            factors = pca.transform(window)
            loadings = pca.components_.T  # Stock weights in each factor
            explained = np.dot(factors, loadings.T)
            # Compute residuals using original returns
            original_window = returns_stocks.iloc[t - n:t]
            residuals.iloc[t] = original_window.iloc[-1] - explained[-1]  # Residual for latest day
            explained_var.iloc[t - n] = sum(pca.explained_variance_ratio_)  # Cumulative explained variance
            eigenvalues_df.iloc[t - n] = pca.explained_variance_  # Eigenvalues
            loadings_df = pd.DataFrame(loadings, index=window.columns, columns=[f'Factor_{i+1}' for i in range(self.n_factors)])
            loadings_df['date'] = window.index[-1]
            loadings_list.append(loadings_df)

        residuals = residuals.dropna()
        loadings_all = pd.concat(loadings_list)
        self.residuals = residuals
        self.pca_metrics = {
            'explained_variance': explained_var,
            'loadings': loadings_all,
            'eigenvalues': eigenvalues_df
        }
        return explained_var, residuals, loadings_all, eigenvalues_df

    def fit_ou_process(self, series: pd.Series, window: int) -> Dict[str, float]:
        """Fit OU process to a residual series using AR(1) model and return OU parameters."""
        if len(series) < window:
            return {'kappa': np.nan, 'm': np.nan, 'sigma': np.nan, 's_score': np.nan}
        
        series_window = series[-window:].dropna()
        if len(series_window) < window:
            return {'kappa': np.nan, 'm': np.nan, 'sigma': np.nan, 's_score': np.nan}
        
        try:
            model = AutoReg(series_window, lags=1).fit()
            a, b = model.params  # Intercept and slope
            if b <= 0 or b >= 1:  # Ensure stationarity
                return {'kappa': np.nan, 'm': np.nan, 'sigma': np.nan, 's_score': np.nan}
            
            # OU parameters
            kappa = -np.log(b)* np.log(252)  # Mean-reversion speed
            m = a / (1 - b)  # Long-term mean
            sigma = np.sqrt(model.sigma2 * 2 * kappa / (1 - b**2))  # Volatility
            
            # S-score
            latest = series.iloc[-1]
            sigma_eq = sigma / np.sqrt(2 * kappa) if kappa > 0 else np.inf
            s_score = (latest - m) / sigma_eq if sigma_eq != 0 else 0
            
            return {'kappa': kappa, 'm': m, 'sigma': sigma, 's_score': s_score}
        
        except (ValueError, np.linalg.LinAlgError):
            return {'kappa': np.nan, 'm': np.nan, 'sigma': np.nan, 's_score': np.nan}
    def apply_ou_fitting(self) -> pd.DataFrame:
        """Apply OU fitting to all residuals and return OU parameters DataFrame."""
        columns = pd.MultiIndex.from_product([self.residuals.columns, ['kappa', 'm', 'sigma', 's_score']])
        self.ou_params = pd.DataFrame(index=self.residuals.index, columns=columns)
        for t in range(self.ou_window, len(self.residuals)):
            date = self.residuals.index[t]
            for stock in self.residuals.columns:
                series = self.residuals[stock].iloc[:t + 1]
                params = self.fit_ou_process(series, self.ou_window)
                for param, value in params.items():
                    self.ou_params.loc[date, (stock, param)] = value
        return self.ou_params

    def generate_signals(self) -> pd.DataFrame:
        """Generate long-only trading signals based on s-scores."""
        self.signals = pd.DataFrame(0, index=self.ou_params.index, columns=self.residuals.columns)
        for date in self.signals.index:
            for stock in self.signals.columns:
                s_score = self.ou_params.loc[date, (stock, 's_score')]
                if pd.notna(s_score):
                    if s_score < -1.25 and self.positions[stock] == 0:
                        self.signals.loc[date, stock] = 1  # Buy signal (only if not holding)
                    elif s_score > -0.5 and self.positions[stock] > 0:
                        self.signals.loc[date, stock] = -1  # Sell signal (only if holding)
        return self.signals

    def open_stock_position(self, date: pd.Timestamp, stock: str, price: float) -> bool:
        allocation = min(self.allocation_per_stock * self.cash, self.cash)
        shares = int(allocation / price)
        if shares <= 0:
            return False
        cost = shares * price * (1 + self.fee_rate)
        if cost <= self.cash:
            self.cash -= cost
            self.positions[stock] = shares
            self.entry_prices[stock] = price
            self.trading_log.append({
                'date': date,
                'stock': stock,
                'action': 'buy',
                'quantity': shares
            })
            return True
        return False

    def close_stock_position(self, date: pd.Timestamp, stock: str, price: float) -> bool:
        shares = self.positions[stock]
        if shares <= 0:
            return False
        proceeds = shares * price * (1 - self.fee_rate)
        self.cash += proceeds
        self.positions[stock] = 0
        self.entry_prices[stock] = 0
        self.trading_log.append({
            'date': date,
            'stock': stock,
            'action': 'sell',
            'quantity': shares
        })
        return True

    def adjust_spy_short(self, date: pd.Timestamp, price_spy: float) -> bool:
        """Adjust SPY short position for beta neutrality; return True if adjusted."""
        V_s = sum(self.positions[stock] * self.data.loc[date, stock]
                  for stock in self.tickers if stock != 'SPY' and self.positions[stock] > 0)
        if V_s == 0:
            shares_short_desired = 0
        else:
            beta_s = sum((self.positions[stock] * self.data.loc[date, stock] / V_s) * self.betas.loc[date, stock]
                         for stock in self.tickers if stock != 'SPY' and self.positions[stock] > 0)
            V_short = beta_s * V_s
            shares_short_desired = int(V_short / price_spy)
        
        current_shares_short = -self.positions['SPY'] if self.positions['SPY'] < 0 else 0
        if shares_short_desired == current_shares_short:
            return False
        
        if self.positions['SPY'] < 0:
            shares_to_close = -self.positions['SPY']
            proceeds = shares_to_close * price_spy * (1 - self.fee_rate)
            self.cash += proceeds + self.margin_posted
            self.margin_posted = 0
            self.positions['SPY'] = 0
            self.trading_log.append({
                'date': date,
                'stock': 'SPY',
                'action': 'buy_to_cover',
                'quantity': shares_to_close
            })
        
        if shares_short_desired > 0:
            margin_new = self.margin_spy * shares_short_desired * price_spy
            cost = shares_short_desired * price_spy * self.fee_rate
            if self.cash >= margin_new + cost:
                self.cash -= (margin_new + cost)
                self.margin_posted = margin_new
                self.positions['SPY'] = -shares_short_desired
                self.entry_prices['SPY'] = price_spy
                self.trading_log.append({
                    'date': date,
                    'stock': 'SPY',
                    'action': 'sell_short',
                    'quantity': shares_short_desired
                })
                return True
        return False
    def run(self) -> Tuple[pd.DataFrame, pd.DataFrame, pd.DataFrame, pd.DataFrame, Dict]:
        prices, returns = self.fetch_data()
        self.calculate_betas()
        explained_var, residuals, loadings, eigenvalues = self.compute_pca_factors()
        self.apply_ou_fitting()
        self.generate_signals()

        for t in range(max(self.pca_window, self.ou_window, self.beta_window), len(self.returns)):
            date = self.returns.index[t]
            if date not in self.signals.index:
                continue
            for stock in self.tickers:
                if stock == 'SPY':
                    continue
                signal = self.signals.loc[date, stock]
                price = self.data.loc[date, stock]
                if signal == 1:
                    self.open_stock_position(date, stock, price)
                elif signal == -1:
                    self.close_stock_position(date, stock, price)
            self.adjust_spy_short(date, self.data.loc[date, 'SPY'])
            
            # Record portfolio state
            positions = {stock: qty for stock, qty in self.positions.items() if qty != 0}
            self.asset_list.append({
                'date': date,
                'balance': self.cash,
                'positions': positions.copy()
            })

        asset_df = pd.DataFrame(self.asset_list)
        trading_log_df = pd.DataFrame(self.trading_log)
        pca_metrics = {
            'explained_variance': explained_var,
            'loadings': loadings,
            'eigenvalues': eigenvalues
        }
        return prices, returns, asset_df, trading_log_df, pca_metrics

In [100]:
class DataProcessor:
    def __init__(self, tickers: List[str], start_date: str, end_date: str, pca_window: int = 252, beta_window: int = 252, n_factors: int = 15):
        self.tickers = tickers + ['SPY']
        self.start_date = start_date
        self.end_date = end_date
        self.pca_window = pca_window
        self.beta_window = beta_window
        self.n_factors = n_factors
        self.data = None
        self.returns = None
        self.standardized_returns = None
        self.betas = None
        self.residuals = None
        self.pca_metrics = {}

    def fetch_data(self):
        """Fetch closing prices and compute returns."""
        data = yf.download(self.tickers, start=self.start_date, end=self.end_date)['Close']
        data = data.dropna()
        self.data = data
        self.returns = data.pct_change().dropna()

    def compute_standardized_returns(self):
        """Compute rolling 60-day standardized returns."""
        returns_stocks = self.returns.drop(columns=['SPY'])
        rolling_mean = returns_stocks.rolling(window=60).mean().shift(1)
        rolling_std = returns_stocks.rolling(window=60).std().shift(1)
        self.standardized_returns = (returns_stocks - rolling_mean) / rolling_std
        self.standardized_returns = self.standardized_returns.dropna()

    def calculate_betas(self):
        """Compute rolling betas relative to SPY."""
        spy_returns = self.returns['SPY']
        betas = pd.DataFrame(index=self.returns.index, columns=[t for t in self.tickers if t != 'SPY'])
        for stock in betas.columns:
            stock_returns = self.returns[stock]
            rolling_cov = stock_returns.rolling(window=self.beta_window).cov(spy_returns)
            rolling_var_spy = spy_returns.rolling(window=self.beta_window).var()
            betas[stock] = rolling_cov / rolling_var_spy
        self.betas = betas.dropna()

    def compute_pca_factors(self):
        """Compute PCA residuals and metrics."""
        returns_stocks = self.returns.drop(columns=['SPY'])
        n = self.pca_window
        residuals = pd.DataFrame(index=returns_stocks.index, columns=returns_stocks.columns)
        explained_var = pd.Series(index=returns_stocks.index[n-1:], dtype=float)
        eigenvalues_df = pd.DataFrame(index=returns_stocks.index[n-1:], columns=[f'Eigen_{i+1}' for i in range(self.n_factors)])
        loadings_list = []

        for t in range(n, len(returns_stocks)):
            window = self.standardized_returns.iloc[t - n:t]
            pca = PCA(n_components=self.n_factors)
            pca.fit(window)
            factors = pca.transform(window)
            loadings = pca.components_.T
            explained = np.dot(factors, loadings.T)
            original_window = returns_stocks.iloc[t - n:t]
            residuals.iloc[t] = original_window.iloc[-1] - explained[-1]
            explained_var.iloc[t - n] = sum(pca.explained_variance_ratio_)
            eigenvalues_df.iloc[t - n] = pca.explained_variance_
            loadings_df = pd.DataFrame(loadings, index=window.columns, columns=[f'Factor_{i+1}' for i in range(self.n_factors)])
            loadings_df['date'] = window.index[-1]
            loadings_list.append(loadings_df)

        self.residuals = residuals.dropna()
        self.pca_metrics = {
            'explained_variance': explained_var,
            'eigenvalues': eigenvalues_df,
            'loadings': pd.concat(loadings_list)
        }

    def process_data(self):
        """Run all data processing steps."""
        self.fetch_data()
        self.compute_standardized_returns()
        self.calculate_betas()
        self.compute_pca_factors()
        return self.data, self.returns, self.standardized_returns, self.betas, self.residuals, self.pca_metrics

In [94]:
class SignalGenerator:
    def __init__(self, residuals: pd.DataFrame, ou_window: int = 60):
        self.residuals = residuals
        self.ou_window = ou_window
        self.ou_params = None
        self.signals = None

    def fit_ou_process(self, series: pd.Series) -> Dict[str, float]:
        """Fit OU process to a residual series."""
        if len(series) < self.ou_window:
            return {'kappa': np.nan, 'm': np.nan, 'sigma': np.nan, 's_score': np.nan}
        series_window = series[-self.ou_window:].dropna()
        if len(series_window) < self.ou_window:
            return {'kappa': np.nan, 'm': np.nan, 'sigma': np.nan, 's_score': np.nan}
        try:
            model = AutoReg(series_window, lags=1).fit()
            a, b = model.params
            if b <= 0 or b >= 1:
                return {'kappa': np.nan, 'm': np.nan, 'sigma': np.nan, 's_score': np.nan}
            kappa = -np.log(b) * 252  # Annualized mean-reversion speed
            m = a / (1 - b)
            sigma = np.sqrt(model.sigma2 * 2 * kappa / (1 - b**2))
            latest = series.iloc[-1]
            sigma_eq = sigma / np.sqrt(2 * kappa) if kappa > 0 else np.inf
            s_score = (latest - m) / sigma_eq if sigma_eq != 0 else 0
            return {'kappa': kappa, 'm': m, 'sigma': sigma, 's_score': s_score}
        except (ValueError, np.linalg.LinAlgError):
            return {'kappa': np.nan, 'm': np.nan, 'sigma': np.nan, 's_score': np.nan}

    def apply_ou_fitting(self):
        """Fit OU process to all residuals."""
        columns = pd.MultiIndex.from_product([self.residuals.columns, ['kappa', 'm', 'sigma', 's_score']])
        self.ou_params = pd.DataFrame(index=self.residuals.index, columns=columns)
        for t in range(self.ou_window, len(self.residuals)):
            date = self.residuals.index[t]
            for stock in self.residuals.columns:
                series = self.residuals[stock].iloc[:t + 1]
                params = self.fit_ou_process(series)
                for param, value in params.items():
                    self.ou_params.loc[date, (stock, param)] = value

    def generate_signals(self, positions: Dict[str, int]):
        """Generate trading signals based on s-scores."""
        self.signals = pd.DataFrame(0, index=self.ou_params.index, columns=self.residuals.columns)
        for date in self.signals.index:
            for stock in self.signals.columns:
                s_score = self.ou_params.loc[date, (stock, 's_score')]
                if pd.notna(s_score):
                    if s_score < -1.25 and positions.get(stock, 0) == 0:
                        self.signals.loc[date, stock] = 1  # Buy
                    elif s_score > -0.5 and positions.get(stock, 0) > 0:
                        self.signals.loc[date, stock] = -1  # Sell

    def generate(self, positions: Dict[str, int]):
        """Run OU fitting and signal generation."""
        self.apply_ou_fitting()
        self.generate_signals(positions)

In [95]:
class PortfolioManager:
    def __init__(self, initial_balance: float = 1000000, fee_rate: float = 0.0043, margin_spy: float = 0.25, allocation_per_stock: float = 0.05):
        self.cash = initial_balance
        self.fee_rate = fee_rate
        self.margin_spy = margin_spy
        self.allocation_per_stock = allocation_per_stock
        self.positions: Dict[str, int] = {}
        self.entry_prices: Dict[str, float] = {}
        self.margin_posted = 0
        self.asset_list = []
        self.trading_log = []

    def open_stock_position(self, date: pd.Timestamp, stock: str, price: float):
        """Open a long position in a stock."""
        allocation = min(self.allocation_per_stock * self.cash, self.cash)
        shares = int(allocation / price)
        if shares <= 0:
            return
        cost = shares * price * (1 + self.fee_rate)
        if cost <= self.cash:
            self.cash -= cost
            self.positions[stock] = shares
            self.entry_prices[stock] = price
            self.trading_log.append({'date': date, 'stock': stock, 'action': 'buy', 'quantity': shares})

    def close_stock_position(self, date: pd.Timestamp, stock: str, price: float):
        """Close a long position in a stock."""
        shares = self.positions.get(stock, 0)
        if shares <= 0:
            return
        proceeds = shares * price * (1 - self.fee_rate)
        self.cash += proceeds
        self.positions[stock] = 0
        self.entry_prices[stock] = 0
        self.trading_log.append({'date': date, 'stock': stock, 'action': 'sell', 'quantity': shares})

    def adjust_spy_short(self, date: pd.Timestamp, price_spy: float, V_s: float, beta_s: float):
        """Adjust SPY short position for beta neutrality."""
        if V_s == 0:
            shares_short_desired = 0
        else:
            V_short = beta_s * V_s
            shares_short_desired = int(V_short / price_spy)

        current_shares_short = -self.positions.get('SPY', 0) if self.positions.get('SPY', 0) < 0 else 0
        if shares_short_desired == current_shares_short:
            return

        if self.positions.get('SPY', 0) < 0:
            shares_to_close = -self.positions['SPY']
            proceeds = shares_to_close * price_spy * (1 - self.fee_rate)
            self.cash += proceeds + self.margin_posted
            self.margin_posted = 0
            self.positions['SPY'] = 0
            self.trading_log.append({'date': date, 'stock': 'SPY', 'action': 'buy_to_cover', 'quantity': shares_to_close})

        if shares_short_desired > 0:
            margin_new = self.margin_spy * shares_short_desired * price_spy
            cost = shares_short_desired * price_spy * self.fee_rate
            if self.cash >= margin_new + cost:
                self.cash -= (margin_new + cost)
                self.margin_posted = margin_new
                self.positions['SPY'] = -shares_short_desired
                self.entry_prices['SPY'] = price_spy
                self.trading_log.append({'date': date, 'stock': 'SPY', 'action': 'sell_short', 'quantity': shares_short_desired})

    def run_trading(self, signals: pd.DataFrame, prices: pd.DataFrame, betas: pd.DataFrame):
        """Execute trades based on signals."""
        for date in signals.index:
            for stock in signals.columns:
                signal = signals.loc[date, stock]
                price = prices.loc[date, stock]
                if signal == 1:
                    self.open_stock_position(date, stock, price)
                elif signal == -1:
                    self.close_stock_position(date, stock, price)

            # Adjust SPY short
            V_s = sum(self.positions[stock] * prices.loc[date, stock] for stock in self.positions if stock != 'SPY' and self.positions[stock] > 0)
            beta_s = sum((self.positions[stock] * prices.loc[date, stock] / V_s) * betas.loc[date, stock] for stock in self.positions if stock != 'SPY' and self.positions[stock] > 0) if V_s > 0 else 0
            self.adjust_spy_short(date, prices.loc[date, 'SPY'], V_s, beta_s)

            # Record portfolio state
            positions = {stock: qty for stock, qty in self.positions.items() if qty != 0}
            self.asset_list.append({'date': date, 'balance': self.cash, 'positions': positions.copy()})

        return pd.DataFrame(self.asset_list), pd.DataFrame(self.trading_log)

In [96]:
tickers = ['AAPL', 'MSFT', 'GOOGL', 'AMZN','META', 'TSLA', 'NVDA', 'AMD', 'INTC', 'CSCO',
           'XOM', 'CVX', 'COP', 'SLB', 'HAL', 'OXY', 'VLO', 'MPC', 'PSX', 'EOG',
           'JPM', 'BAC', 'WFC', 'C', 'GS', 'MS', 'PFE', 'JNJ', 'MRK', 'LLY',
           'PG', 'KO', 'PEP', 'WMT', 'TGT', 'COST', 'HD', 'LOW', 'DIS', 'NFLX',
           ]
start_date = "2020-01-01"
end_date="2022-12-31"

In [102]:
# Initialize classes
data_processor = DataProcessor(tickers, start_date, end_date)
signal_generator = SignalGenerator(None)  # Residuals assigned after processing
portfolio_manager = PortfolioManager()

In [103]:
# Process data
data, returns, standardized_returns, betas, residuals, pca_metrics=data_processor.process_data()

[*********************100%***********************]  41 of 41 completed


In [104]:
residuals

Ticker,AAPL,AMD,AMZN,BAC,C,COP,COST,CSCO,CVX,DIS,...,PFE,PG,PSX,SLB,TGT,TSLA,VLO,WFC,WMT,XOM
Date,,,,,,,,,,,,,,,,,,,,,
2021-01-04,0.50331,0.377758,0.337889,-0.487069,-0.54593,0.414488,0.7536,1.163291,0.435145,-0.154415,...,1.528503,1.375432,0.431423,0.091519,-0.042356,-0.714521,0.32575,-0.769802,0.70152,0.485696
2021-01-05,-1.000296,-1.408681,-0.918396,0.881565,0.813641,0.538678,-0.514536,0.253203,0.66836,0.328426,...,-0.363921,0.371253,0.560267,0.540321,0.388442,-0.932729,0.468899,0.880283,-0.128957,0.711144
2021-01-06,-0.602764,-1.449358,-0.922303,-0.292788,-0.295883,-0.512361,-0.237728,0.080588,-0.543955,-1.286876,...,-0.227481,0.689105,-1.036405,-0.780752,-0.567167,0.0713,-1.305271,-0.214103,-0.104644,-0.654039
2021-01-07,-1.536017,-0.184648,-1.20289,0.39461,0.436657,1.261968,-1.786217,-0.685149,1.013455,-0.525169,...,-0.045176,-1.422793,1.156631,1.052088,-0.723886,-0.482696,1.294426,0.561712,-1.890288,1.142114
2021-01-08,0.356711,0.170654,0.042636,-0.021263,-0.098318,0.229431,-0.050566,0.369105,0.22262,-0.010252,...,1.050057,-0.204342,0.472132,0.230903,0.258734,-0.09791,0.52195,-0.002888,-0.33318,0.292328
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-12-23,-0.032214,-0.25116,-0.140122,-0.072617,0.011923,-0.470623,0.089129,-0.104233,-0.407124,-0.006231,...,0.183333,0.649664,-0.099897,-0.296865,-0.266621,-0.650286,-0.098513,0.10341,0.139371,-0.307912
2022-12-27,-0.009643,-0.193131,-0.09197,-0.061552,0.031458,-0.395743,0.105818,-0.099544,-0.359704,0.001395,...,0.180141,0.652835,-0.035481,-0.223371,-0.251723,-0.572357,-0.031424,0.119974,0.151782,-0.258664
2022-12-28,-0.012631,-0.217536,-0.1295,-0.057393,0.032751,-0.427606,0.093312,-0.104656,-0.382085,-0.025964,...,0.157528,0.661766,-0.066879,-0.246589,-0.239645,-0.670104,-0.056775,0.114732,0.152245,-0.275163
